In [1]:
# 기틀 json에서 img src를 실제 저장 경로로 바꾸기 
#img src=\"detailed_docs/images/26343397/img_1762410725517_1.png\" title=\"external_image\" alt=\"external_image\" 

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
JSON 파일의 img src 경로를 CSV의 saved_as 경로로 바꾸는 스크립트
"""

import json
import csv
import re
import sys  
from pathlib import Path

csv.field_size_limit(min(sys.maxsize, 2147483647))  
def load_json_file(json_path):
    """JSON 파일을 로드합니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)


def load_csv_mappings(csv_path):
    """
    CSV 파일에서 이미지 src와 saved_as 매핑을 추출합니다.
    
    Returns:
        dict: {image_id: saved_as_path} 형태의 매핑 딕셔너리
    """
    mappings = {}
    
    with open(csv_path, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            # '내용' 컬럼에서 JSON 파싱
            try:
                content = json.loads(row['내용'])
                
                # body_images에서 이미지 매핑 추출
                if '본문' in content and 'body_images' in content['본문']:
                    for img in content['본문']['body_images']:
                        src = img.get('src', '')
                        saved_as = img.get('saved_as', '')
                        
                        # src에서 이미지 ID 추출 (마지막 부분)
                        # 예: http://office.anyfive.com/image/namoimage/20251020/b5f5dc3c-...
                        # -> /image/namoimage/20251020/b5f5dc3c-...
                        if src:
                            # URL에서 경로 부분만 추출
                            path_match = re.search(r'/image/namoimage/\d+/[a-f0-9-]+', src)
                            if path_match:
                                img_path = path_match.group(0)
                                mappings[img_path] = saved_as
            except json.JSONDecodeError:
                continue
    
    return mappings


def replace_img_paths_in_json(json_data, mappings):
    """
    JSON 데이터의 docBody에서 img src 경로를 saved_as 경로로 바꿉니다.
    
    Args:
        json_data: JSON 데이터 (리스트 또는 딕셔너리)
        mappings: {image_path: saved_as_path} 형태의 매핑 딕셔너리
    
    Returns:
        수정된 JSON 데이터와 교체 통계
    """
    replaced_count = 0
    not_found_count = 0
    not_found_paths = []
    
    # JSON 데이터가 리스트인 경우
    if isinstance(json_data, list):
        for item in json_data:
            if 'docBody' in item:
                doc_body = item['docBody']
                
                # img src 패턴 찾기: src="/image/namoimage/..."
                img_pattern = r'src="(/image/namoimage/\d+/[a-f0-9-]+)"'
                
                def replace_func(match):
                    nonlocal replaced_count, not_found_count
                    img_path = match.group(1)
                    
                    if img_path in mappings:
                        replaced_count += 1
                        # 백슬래시를 슬래시로 변환 (경로 정규화)
                        saved_path = mappings[img_path].replace('\\', '/')
                        return f'src="{saved_path}"'
                    else:
                        not_found_count += 1
                        not_found_paths.append(img_path)
                        return match.group(0)  # 원본 유지
                
                # 정규식으로 모든 img src 교체
                item['docBody'] = re.sub(img_pattern, replace_func, doc_body)
    
    return json_data, {
        'replaced': replaced_count,
        'not_found': not_found_count,
        'not_found_paths': not_found_paths
    }


def main():
    """메인 함수"""
    # 파일 경로 설정
    json_path = '결재목록_교체완료.json'
    csv_path = '전자결재_목록.csv'
    output_path = 'any_docBodyImageSrcChange.json'
    
    print("=" * 60)
    print("JSON 파일의 img src 경로를 CSV의 saved_as 경로로 변경")
    print("=" * 60)
    
    # 1. JSON 파일 로드
    print("\n1. JSON 파일 로드 중...")
    json_data = load_json_file(json_path)
    print(f"   ✓ {len(json_data)}개의 문서를 로드했습니다.")
    
    # 2. CSV에서 이미지 경로 매핑 추출
    print("\n2. CSV에서 이미지 경로 매핑 추출 중...")
    mappings = load_csv_mappings(csv_path)
    print(f"   ✓ {len(mappings)}개의 이미지 매핑을 찾았습니다.")
    
    if mappings:
        print("\n   매핑 예시:")
        for i, (key, value) in enumerate(list(mappings.items())[:3], 1):
            print(f"   {i}. {key}")
            print(f"      -> {value}")
    
    # 3. JSON 데이터에서 img src 경로 교체
    print("\n3. JSON 데이터에서 img src 경로 교체 중...")
    modified_data, stats = replace_img_paths_in_json(json_data, mappings)
    
    print(f"   ✓ 교체 완료: {stats['replaced']}개")
    print(f"   ⚠ 매칭 실패: {stats['not_found']}개")
    
    if stats['not_found_paths']:
        print("\n   매칭 실패한 경로:")
        for path in stats['not_found_paths'][:5]:
            print(f"   - {path}")
        if len(stats['not_found_paths']) > 5:
            print(f"   ... 외 {len(stats['not_found_paths']) - 5}개")
    
    # 4. 결과 저장
    print("\n4. 결과 파일 저장 중...")
    output_dir = Path(output_path).parent
    output_dir.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(modified_data, f, ensure_ascii=False, indent=2)
    
    print(f"   ✓ 결과 파일 저장 완료: {output_path}")
    
    print("\n" + "=" * 60)
    print("작업 완료!")
    print("=" * 60)
    
    return stats


if __name__ == "__main__":
    main()

JSON 파일의 img src 경로를 CSV의 saved_as 경로로 변경

1. JSON 파일 로드 중...
   ✓ 3개의 문서를 로드했습니다.

2. CSV에서 이미지 경로 매핑 추출 중...
   ✓ 0개의 이미지 매핑을 찾았습니다.

3. JSON 데이터에서 img src 경로 교체 중...
   ✓ 교체 완료: 0개
   ⚠ 매칭 실패: 0개

4. 결과 파일 저장 중...
   ✓ 결과 파일 저장 완료: any_docBodyImageSrcChange.json

작업 완료!
